In [108]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np



In [109]:
currency_list = ['USDCHF10080',
                 'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]

all_currency_prediction = []

for q in currency_list:

    currency = q.replace('10080','')
    
    data = pd.read_excel('files/currency_training_data/' + currency +'_combine_data_dataframe.xlsx', sheet_name=0)
    
    # classify each row and attach numeric score to it
    classification_score = []

    # Iterate over each row
    for index, row in data.iterrows():

        # condition when the curreny movement is a buy

        pips_corrector = 100000

        if currency.endswith('jpy'):# use this to correct multiplier of jpy pairs
            pips_corrector = 1000

        if row.nextweek_open < row.nextweek_close:

            trend = 1


            #classifying the risk level


            if (row.nextweek_open - row.nextweek_low) == 0 : # no risk because open is equal to low

                risk = 6

            else : 

                if (row.nextweek_open - row.nextweek_low)> (row.nextweek_high - row.nextweek_open):

                    risk = 1

                if  (row.nextweek_high - row.nextweek_open) > (row.nextweek_open - row.nextweek_low):

                    risk = 2


                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >2 :

                    risk = 3

                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >4 :

                    risk = 4


                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >8 :

                    risk = 5

                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >16 :

                    risk = 6  

            #classifying reward level

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 0 :

                reward = 1

            if (row.nextweek_close - row.nextweek_open)*pips_corrector >200 :

                reward = 2

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 400 :

                reward = 3


            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 800 :

                reward = 4

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 1600 :

                reward = 5


            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 3200 :

                reward = 6  




        # this is for the condition when the movement is a sell
        else: 

            trend = -1

            #classifying the risk level


            if (row.nextweek_high - row.nextweek_open) == 0 : # no risk because open is equal to high

                risk = 6

            else : 

                if (row.nextweek_high - row.nextweek_open)> (row.nextweek_open - row.nextweek_low):

                    risk = 1

                if  (row.nextweek_open - row.nextweek_low) > (row.nextweek_high - row.nextweek_open):

                    risk = 2


                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >2 :

                    risk = 3

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >4 :

                    risk = 4

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >8 :

                    risk = 5

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >16 :

                    risk = 6 


             #classifying reward level

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 0 :

                reward = 1

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 200 :

                reward = 2

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 400 :

                reward = 3        

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 800 :

                reward = 4    

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 1600 :

                reward = 5 

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 3200 :

                reward = 6        

        classification_score.append(trend*risk*reward)



    #attach the classification score value to the dataframe and train a model with the data


    
    data['classification_score'] = classification_score 
    
    



    X = data.drop(columns=['Unnamed: 0', 
                           'date_start',  'nextweek_open', 'nextweek_high','nextweek_low',                       
                            'nextweek_close','classification_score',


                          ])




   

    y = data['classification_score']
    
   
  
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.00001 )

    model = LinearRegression()
    model.fit(X, y)

    
    
    temp_data = pd.read_excel('files/combine_top_10_rows_dataframe.xlsx', sheet_name=0)
    
    #temp_data = temp_data.head(1)
    
    
    
    #currency1 =q[ 0 : 3 ].lower()
    #currency2 = q[3:6].lower()
    
   
    
    #index1 = temp_data.columns.get_loc(currency1+'_buy_percent')
    #index2 = temp_data.columns.get_loc(currency2+'_buy_percent')
    
    #temp_data = temp_data.iloc[:, [ 1, index1, index1+1, index1+2, index1+3, index2, index2+1, index2+2, index2+3]   ]
    
    
    
    #loads corresponding historical price data so as to get currency volume and add it to the data
    #headers = ['date_start', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]
    #current_currency_volume = pd.read_csv('recent_data/' + q + '.csv', names = headers)
    
    #for index, row in current_currency_volume.iterrows():
       
        #if row.date_start.replace(".", "-") == temp_data['date_start'][0].strftime('%Y-%m-%d'):
          
            #temp_data.insert(0, 'volume', [row.volume])
            
    
    
    last_date = temp_data['date_start'][0].strftime('%Y-%m-%d')
    X = temp_data.drop(columns=[ 'date_start','Unnamed: 0' ])
   
    
 
    y_pred = model.predict(X)
    all_currency_prediction.append(y_pred[0])
    
    print(q)
    
print(all_currency_prediction)


USDCHF10080
GBPUSD10080
EURUSD10080
USDJPY10080
USDCAD10080
AUDUSD10080
NZDUSD10080
GBPCHF10080
EURCHF10080
CHFJPY10080
CADCHF10080
AUDCHF10080
NZDCHF10080
EURGBP10080
GBPJPY10080
GBPCAD10080
GBPAUD10080
EURJPY10080
EURCAD10080
EURAUD10080
EURNZD10080
CADJPY10080
AUDJPY10080
NZDJPY10080
AUDCAD10080
NZDCAD10080
AUDNZD10080
[0.6926084422742842, 5.173630164097615, 1.3167374378006294, 4.141800394155128, -1.0955590767568797, 1.6769145689429417, 0.42154720931525447, 5.857749167573818, 1.4687031567993771, 10.84155550874957, 3.145849097526351, 3.4034150669566894, 1.6783392974685092, -1.8751217584832975, 10.417729646084766, 4.531792427638322, 4.996562657379496, 6.37185921025773, 0.4728724468061285, -0.009184681093269198, 1.5754497690672629, 6.46424135313341, 7.4183187238251795, 4.222429397361259, 1.4674268541346045, 0.9922826702590739, 3.931394927652031]


In [110]:
 data = {
            'currency':currency_list,
            'score':all_currency_prediction
     
             }
                                     
df = pd.DataFrame(data)

df.to_excel('files/' + last_date + '_' + 'scores.xlsx')